In [ ]:
import sqlite3
import pandas as pd

conexion = sqlite3.connect("NBA_DATA.db")

cursor = conexion.cursor()

# Crear la tabla PLAY_BY_PLAY
cursor.execute("""
CREATE TABLE IF NOT EXISTS "PLAY_BY_PLAY" (
  "GAME_ID" TEXT,
  "EVENTNUM" INTEGER,
  "EVENTMSGTYPE" INTEGER,
  "EVENTMSGACTIONTYPE" INTEGER,
  "PERIOD" INTEGER,
  "WCTIMESTRING" TEXT,
  "PCTIMESTRING" TEXT,
  "HOMEDESCRIPTION" TEXT,
  "NEUTRALDESCRIPTION" TEXT,
  "VISITORDESCRIPTION" TEXT,
  "SCORE" TEXT,
  "SCOREMARGIN" TEXT,
  "PERSON1TYPE" INTEGER,
  "PLAYER1_ID" INTEGER,
  "PLAYER1_NAME" TEXT,
  "PLAYER1_TEAM_ID" REAL,
  "PLAYER1_TEAM_CITY" TEXT,
  "PLAYER1_TEAM_NICKNAME" TEXT,
  "PLAYER1_TEAM_ABBREVIATION" TEXT,
  "PERSON2TYPE" INTEGER,
  "PLAYER2_ID" INTEGER,
  "PLAYER2_NAME" TEXT,
  "PLAYER2_TEAM_ID" REAL,
  "PLAYER2_TEAM_CITY" TEXT,
  "PLAYER2_TEAM_NICKNAME" TEXT,
  "PLAYER2_TEAM_ABBREVIATION" TEXT,
  "PERSON3TYPE" INTEGER,
  "PLAYER3_ID" INTEGER,
  "PLAYER3_NAME" TEXT,
  "PLAYER3_TEAM_ID" REAL,
  "PLAYER3_TEAM_CITY" TEXT,
  "PLAYER3_TEAM_NICKNAME" TEXT,
  "PLAYER3_TEAM_ABBREVIATION" TEXT,
  "VIDEO_AVAILABLE_FLAG" INTEGER
)
""")

# Crear la tabla GAME_SUMMARY
cursor.execute("""
CREATE TABLE IF NOT EXISTS "GAME_SUMMARY" (
  "GAME_DATE_EST" TEXT,
  "GAME_SEQUENCE" INTEGER,
  "GAME_ID" TEXT,
  "GAME_STATUS_ID" INTEGER,
  "GAME_STATUS_TEXT" TEXT,
  "GAMECODE" TEXT,
  "HOME_TEAM_ID" INTEGER,
  "VISITOR_TEAM_ID" INTEGER,
  "SEASON" TEXT,
  "LIVE_PERIOD" INTEGER,
  "LIVE_PC_TIME" TEXT,
  "NATL_TV_BROADCASTER_ABBREVIATION" TEXT,
  "LIVE_PERIOD_TIME_BCAST" TEXT,
  "WH_STATUS" INTEGER
)
""")

# Crear la tabla TEAM_INFO_COMMON
cursor.execute("""
CREATE TABLE IF NOT EXISTS "TEAM_INFO_COMMON" (
"TEAM_ID" INTEGER,
  "SEASON_YEAR" TEXT,
  "TEAM_CITY" TEXT,
  "TEAM_NAME" TEXT,
  "TEAM_ABBREVIATION" TEXT,
  "TEAM_CONFERENCE" TEXT,
  "TEAM_DIVISION" TEXT,
  "TEAM_CODE" TEXT,
  "TEAM_SLUG" TEXT,
  "W" INTEGER,
  "L" INTEGER,
  "PCT" REAL,
  "CONF_RANK" INTEGER,
  "DIV_RANK" INTEGER,
  "MIN_YEAR" TEXT,
  "MAX_YEAR" TEXT
)
""")

conexion.close()

In [27]:
from nba_api.stats.endpoints import playbyplayv2, BoxScoreSummaryV2, TeamInfoCommon
import pandas as pd

# ID de un partido reciente
game_id = "0022400256"  # Indiana Pacers vs. New Orleans Pelicans, 25 de noviembre de 2024

# Llamar al endpoint
# Llamada al endpoint BoxScoreSummaryV2
box_score = BoxScoreSummaryV2(game_id=game_id)

# Extraer datos como DataFrame
pbp_data = box_score.game_summary.get_data_frame()

home_team_id = pbp_data['HOME_TEAM_ID'][0]
team_info = TeamInfoCommon(team_id=home_team_id).team_info_common.get_data_frame()


conexion = sqlite3.connect("mi_base_de_datos.db")
cursor = conexion.cursor()

# Crear la tabla a partir del DataFrame
team_info.to_sql("TEAM_INFO_COMMON", conexion, if_exists="replace", index=False)

# Consultar la declaración SQL de la tabla
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='TEAM_INFO_COMMON'")
declaracion = cursor.fetchone()

# Mostrar la declaración SQL
print(declaracion[0])

# Cerrar la conexión
conexion.close()

CREATE TABLE "TEAM_INFO_COMMON" (
"TEAM_ID" INTEGER,
  "SEASON_YEAR" TEXT,
  "TEAM_CITY" TEXT,
  "TEAM_NAME" TEXT,
  "TEAM_ABBREVIATION" TEXT,
  "TEAM_CONFERENCE" TEXT,
  "TEAM_DIVISION" TEXT,
  "TEAM_CODE" TEXT,
  "TEAM_SLUG" TEXT,
  "W" INTEGER,
  "L" INTEGER,
  "PCT" REAL,
  "CONF_RANK" INTEGER,
  "DIV_RANK" INTEGER,
  "MIN_YEAR" TEXT,
  "MAX_YEAR" TEXT
)


In [26]:
from nba_api.stats.endpoints import playbyplayv2, BoxScoreSummaryV2,TeamInfoCommon
import pandas as pd

# ID de un partido reciente
game_id = "0022400256"  # Indiana Pacers vs. New Orleans Pelicans, 25 de noviembre de 2024

# Llamar al endpoint
# Llamada al endpoint BoxScoreSummaryV2
box_score = BoxScoreSummaryV2(game_id=game_id)

# Extraer datos como DataFrame
pbp_data = box_score.game_summary.get_data_frame()

home_team_id = pbp_data['HOME_TEAM_ID'][0]
team_info = TeamInfoCommon(team_id=home_team_id).team_info_common.get_data_frame()
team_info.columns

Index(['TEAM_ID', 'SEASON_YEAR', 'TEAM_CITY', 'TEAM_NAME', 'TEAM_ABBREVIATION',
       'TEAM_CONFERENCE', 'TEAM_DIVISION', 'TEAM_CODE', 'TEAM_SLUG', 'W', 'L',
       'PCT', 'CONF_RANK', 'DIV_RANK', 'MIN_YEAR', 'MAX_YEAR'],
      dtype='object')

In [28]:
conexion = sqlite3.connect("mi_base_de_datos.db")
cursor = conexion.cursor()

# Obtener la lista de todas las tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tablas = cursor.fetchall()

# Mostrar las tablas
for tabla in tablas:
    print(tabla[0])

# Cerrar la conexión
conexion.close()

PLAY_BY_PLAY
GAME_SUMMARY
TEAM_INFO_COMMON


In [31]:
from nba_api.stats.endpoints import leaguegamefinder

gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable="2023-24")
games = gamefinder.get_data_frames()[0]

# Filtrar partidos de la temporada regular (por fechas)
inicio_temporada = "2023-10-24"
fin_temporada = "2024-04-14"
games = games[(games["GAME_DATE"] >= inicio_temporada) & (games["GAME_DATE"] <= fin_temporada)]
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
178,22023,1610612740,NOP,New Orleans Pelicans,0022301195,2024-04-14,NOP vs. LAL,L,240,108,...,0.786,10,29,39,26,8,1,19,17,-16.0
179,22023,1610612758,SAC,Sacramento Kings,0022301200,2024-04-14,SAC vs. POR,W,241,121,...,0.815,13,38,51,29,11,6,14,19,39.0
180,22023,1610612762,UTA,Utah Jazz,0022301198,2024-04-14,UTA @ GSW,L,240,116,...,0.625,14,34,48,22,6,5,19,20,-7.0
181,22023,1610612749,MIL,Milwaukee Bucks,0022301191,2024-04-14,MIL @ ORL,L,238,88,...,0.913,7,27,34,16,10,4,17,18,-25.0
182,22023,1610612766,CHA,Charlotte Hornets,0022301187,2024-04-14,CHA @ CLE,W,241,120,...,0.818,10,37,47,36,10,9,9,11,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,22023,1610612754,IND,Indiana Pacers,0022300064,2023-10-25,IND vs. WAS,W,240,143,...,0.917,10,41,51,38,10,8,12,23,23.0
4228,22023,1610612744,GSW,Golden State Warriors,0022300062,2023-10-24,GSW vs. PHX,L,240,104,...,0.786,18,31,49,19,11,6,11,23,-4.0
4229,22023,1610612756,PHX,Phoenix Suns,0022300062,2023-10-24,PHX @ GSW,W,241,108,...,0.765,17,43,60,23,5,7,19,22,4.0
4230,22023,1610612743,DEN,Denver Nuggets,0022300061,2023-10-24,DEN vs. LAL,W,240,119,...,0.750,9,33,42,29,9,6,11,15,12.0
